In [1]:
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

In [2]:
root_data_folder = "data"
raw_data_folder = os.path.join(root_data_folder, "raw")
processed_data_folder = os.path.join(root_data_folder, "processed")

In [3]:
stores_sales_df = pd.read_parquet(
    os.path.join(processed_data_folder, "stores-sales.parquet")
)[["Store", "StateHoliday", "Promo", "Sales"]]

stores_sales_df["StateHoliday"] = stores_sales_df["StateHoliday"].map(
    {"0": 0, "a": 1, "b": 2, "c": 3}
)

stores_sales_df

,Store,StateHoliday,Promo,Sales
Date,,,,
2013-01-01,1115,1,0,0
2013-01-01,746,1,0,0
2013-01-01,171,1,0,0
2013-01-01,694,1,0,0
2013-01-01,396,1,0,0
...,...,...,...,...
2015-07-31,612,0,1,8161
2015-07-31,235,0,1,6756
2015-07-31,1078,0,1,9732


In [4]:
grouped_sales_df = stores_sales_df.groupby("Store")

weekly_stores_sales_df = grouped_sales_df[["StateHoliday", "Promo"]].resample("W").max()
weekly_stores_sales_df["Sales"] = grouped_sales_df["Sales"].resample("W").max()

weekly_stores_sales_df = weekly_stores_sales_df.reset_index()
weekly_stores_sales_df

,Store,Date,StateHoliday,Promo,Sales
0,1,2013-01-06,1.0,0.0,5530.0
1,1,2013-01-13,0.0,1.0,7176.0
2,1,2013-01-20,0.0,0.0,5182.0
3,1,2013-01-27,0.0,1.0,5720.0
4,1,2013-02-03,0.0,0.0,5970.0
...,...,...,...,...,...
150520,1115,2015-07-05,0.0,1.0,11006.0
150521,1115,2015-07-12,0.0,0.0,7164.0
150522,1115,2015-07-19,0.0,1.0,10598.0
150523,1115,2015-07-26,0.0,0.0,6897.0


In [5]:
weekly_stores_sales_df["Year"] = weekly_stores_sales_df["Date"].dt.year
weekly_stores_sales_df["Month"] = weekly_stores_sales_df["Date"].dt.month
weekly_stores_sales_df["WeekOfYear"] = weekly_stores_sales_df["Date"].dt.isocalendar().week

weekly_stores_sales_df

,Store,Date,StateHoliday,Promo,Sales,Year,Month,WeekOfYear
0,1,2013-01-06,1.0,0.0,5530.0,2013,1,1
1,1,2013-01-13,0.0,1.0,7176.0,2013,1,2
2,1,2013-01-20,0.0,0.0,5182.0,2013,1,3
3,1,2013-01-27,0.0,1.0,5720.0,2013,1,4
4,1,2013-02-03,0.0,0.0,5970.0,2013,2,5
...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,0.0,1.0,11006.0,2015,7,27
150521,1115,2015-07-12,0.0,0.0,7164.0,2015,7,28
150522,1115,2015-07-19,0.0,1.0,10598.0,2015,7,29
150523,1115,2015-07-26,0.0,0.0,6897.0,2015,7,30


In [6]:
sales_transform = StandardScaler()

weekly_stores_sales_df["Sales"] = sales_transform.fit_transform(weekly_stores_sales_df[["Sales"]]).squeeze()
weekly_stores_sales_df = pd.get_dummies(weekly_stores_sales_df, columns=["StateHoliday"])
weekly_stores_sales_df

,Store,Date,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0
0,1,2013-01-06,0.0,-0.875915,2013,1,1,False,True,False,False
1,1,2013-01-13,1.0,-0.423819,2013,1,2,True,False,False,False
2,1,2013-01-20,0.0,-0.971498,2013,1,3,True,False,False,False
3,1,2013-01-27,1.0,-0.823729,2013,1,4,True,False,False,False
4,1,2013-02-03,0.0,-0.755063,2013,2,5,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,1.0,0.628142,2015,7,27,True,False,False,False
150521,1115,2015-07-12,0.0,-0.427115,2015,7,28,True,False,False,False
150522,1115,2015-07-19,1.0,0.516080,2015,7,29,True,False,False,False
150523,1115,2015-07-26,0.0,-0.500450,2015,7,30,True,False,False,False


In [7]:
weekly_stores_sales_df["Sales_Lag1"] = weekly_stores_sales_df.groupby("Store")["Sales"].shift(1)
weekly_stores_sales_df["Sales_Lag2"] = weekly_stores_sales_df.groupby("Store")["Sales"].shift(2)

weekly_stores_sales_df

,Store,Date,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0,Sales_Lag1,Sales_Lag2
0,1,2013-01-06,0.0,-0.875915,2013,1,1,False,True,False,False,NaN,NaN
1,1,2013-01-13,1.0,-0.423819,2013,1,2,True,False,False,False,-0.875915,NaN
2,1,2013-01-20,0.0,-0.971498,2013,1,3,True,False,False,False,-0.423819,-0.875915
3,1,2013-01-27,1.0,-0.823729,2013,1,4,True,False,False,False,-0.971498,-0.423819
4,1,2013-02-03,0.0,-0.755063,2013,2,5,True,False,False,False,-0.823729,-0.971498
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,1.0,0.628142,2015,7,27,True,False,False,False,-0.561151,0.586119
150521,1115,2015-07-12,0.0,-0.427115,2015,7,28,True,False,False,False,0.628142,-0.561151
150522,1115,2015-07-19,1.0,0.516080,2015,7,29,True,False,False,False,-0.427115,0.628142
150523,1115,2015-07-26,0.0,-0.500450,2015,7,30,True,False,False,False,0.516080,-0.427115


In [8]:
weekly_stores_sales_df["Sales_2Week_Avg"] = weekly_stores_sales_df.groupby("Store")["Sales"].rolling(window=2).mean().reset_index(drop=True)

weekly_stores_sales_df

,Store,Date,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0,Sales_Lag1,Sales_Lag2,Sales_2Week_Avg
0,1,2013-01-06,0.0,-0.875915,2013,1,1,False,True,False,False,NaN,NaN,NaN
1,1,2013-01-13,1.0,-0.423819,2013,1,2,True,False,False,False,-0.875915,NaN,-0.649867
2,1,2013-01-20,0.0,-0.971498,2013,1,3,True,False,False,False,-0.423819,-0.875915,-0.697658
3,1,2013-01-27,1.0,-0.823729,2013,1,4,True,False,False,False,-0.971498,-0.423819,-0.897613
4,1,2013-02-03,0.0,-0.755063,2013,2,5,True,False,False,False,-0.823729,-0.971498,-0.789396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,1.0,0.628142,2015,7,27,True,False,False,False,-0.561151,0.586119,0.033496
150521,1115,2015-07-12,0.0,-0.427115,2015,7,28,True,False,False,False,0.628142,-0.561151,0.100514
150522,1115,2015-07-19,1.0,0.516080,2015,7,29,True,False,False,False,-0.427115,0.628142,0.044482
150523,1115,2015-07-26,0.0,-0.500450,2015,7,30,True,False,False,False,0.516080,-0.427115,0.007815


In [9]:
weekly_stores_sales_df = weekly_stores_sales_df.dropna()
weekly_stores_sales_df

,Store,Date,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0,Sales_Lag1,Sales_Lag2,Sales_2Week_Avg
2,1,2013-01-20,0.0,-0.971498,2013,1,3,True,False,False,False,-0.423819,-0.875915,-0.697658
3,1,2013-01-27,1.0,-0.823729,2013,1,4,True,False,False,False,-0.971498,-0.423819,-0.897613
4,1,2013-02-03,0.0,-0.755063,2013,2,5,True,False,False,False,-0.823729,-0.971498,-0.789396
5,1,2013-02-10,1.0,-0.463370,2013,2,6,True,False,False,False,-0.755063,-0.823729,-0.609217
6,1,2013-02-17,0.0,-0.704525,2013,2,7,True,False,False,False,-0.463370,-0.755063,-0.583948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,1.0,0.628142,2015,7,27,True,False,False,False,-0.561151,0.586119,0.033496
150521,1115,2015-07-12,0.0,-0.427115,2015,7,28,True,False,False,False,0.628142,-0.561151,0.100514
150522,1115,2015-07-19,1.0,0.516080,2015,7,29,True,False,False,False,-0.427115,0.628142,0.044482
150523,1115,2015-07-26,0.0,-0.500450,2015,7,30,True,False,False,False,0.516080,-0.427115,0.007815


In [10]:
suffled_weekly_stores_sales_df = weekly_stores_sales_df.copy()

suffled_weekly_stores_sales_df["rnd_index"] = np.random.default_rng(seed=42).permutation(
    len(suffled_weekly_stores_sales_df)
)

suffled_weekly_stores_sales_df = (
    suffled_weekly_stores_sales_df
        .set_index("rnd_index")
        .sort_index()
        .drop("Date", axis="columns")
)

suffled_weekly_stores_sales_df

,Store,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0,Sales_Lag1,Sales_Lag2,Sales_2Week_Avg
rnd_index,,,,,,,,,,,,,
0,1081,1.0,-0.474082,2015,4,16,True,False,False,False,-1.096195,-0.220842,-0.785139
1,834,0.0,-0.468314,2013,12,48,True,False,False,False,0.100514,-0.836913,-0.183900
2,470,1.0,2.725747,2013,5,18,False,True,False,False,2.156919,1.345563,2.441333
3,460,1.0,-0.276324,2014,9,36,True,False,False,False,-1.275550,-0.488365,-0.775937
4,985,0.0,-0.233477,2014,4,15,True,False,False,False,1.011023,0.158468,0.388773
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143430,870,0.0,-0.344441,2013,12,48,True,False,False,False,-0.124710,-0.677608,-0.234576
143431,88,0.0,0.998114,2013,12,52,False,False,False,True,2.144010,-0.493583,1.571062
143432,549,0.0,-1.101963,2015,7,30,True,False,False,False,-0.781156,-0.861083,-0.941559


In [11]:
data_len = len(suffled_weekly_stores_sales_df.index)
test_len = int(data_len * 0.2)

train_df = suffled_weekly_stores_sales_df.iloc[:-test_len]
test_df = suffled_weekly_stores_sales_df.iloc[-test_len:]

display(train_df)
display(test_df)

,Store,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0,Sales_Lag1,Sales_Lag2,Sales_2Week_Avg
rnd_index,,,,,,,,,,,,,
0,1081,1.0,-0.474082,2015,4,16,True,False,False,False,-1.096195,-0.220842,-0.785139
1,834,0.0,-0.468314,2013,12,48,True,False,False,False,0.100514,-0.836913,-0.183900
2,470,1.0,2.725747,2013,5,18,False,True,False,False,2.156919,1.345563,2.441333
3,460,1.0,-0.276324,2014,9,36,True,False,False,False,-1.275550,-0.488365,-0.775937
4,985,0.0,-0.233477,2014,4,15,True,False,False,False,1.011023,0.158468,0.388773
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114743,672,1.0,0.711915,2015,5,18,False,True,False,False,-0.090652,0.413905,0.310631
114744,589,1.0,-0.085433,2014,4,14,True,False,False,False,-1.010500,-0.060439,-0.547967
114745,304,1.0,-0.019239,2014,7,29,True,False,False,False,-0.413931,1.106606,-0.216585


,Store,Promo,Sales,Year,Month,WeekOfYear,StateHoliday_0.0,StateHoliday_1.0,StateHoliday_2.0,StateHoliday_3.0,Sales_Lag1,Sales_Lag2,Sales_2Week_Avg
rnd_index,,,,,,,,,,,,,
114748,782,1.0,-0.624048,2013,11,47,True,False,False,False,-0.991548,-0.353230,-0.807798
114749,394,1.0,0.238120,2014,9,38,True,False,False,False,-0.069778,0.720979,0.084171
114750,645,0.0,0.679779,2014,3,9,True,False,False,False,1.569414,0.277397,1.124597
114751,112,0.0,-0.358449,2013,5,19,False,True,False,False,-0.383443,-0.548241,-0.370946
114752,823,1.0,0.157644,2014,4,16,False,False,True,False,-0.734188,-0.342244,-0.288272
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143430,870,0.0,-0.344441,2013,12,48,True,False,False,False,-0.124710,-0.677608,-0.234576
143431,88,0.0,0.998114,2013,12,52,False,False,False,True,2.144010,-0.493583,1.571062
143432,549,0.0,-1.101963,2015,7,30,True,False,False,False,-0.781156,-0.861083,-0.941559


In [12]:
X_train, y_train = train_df.drop("Sales", axis="columns"), train_df[["Sales"]]
X_test, y_test = test_df.drop("Sales", axis="columns"), test_df[["Sales"]]

In [13]:
xgbr_regressor = XGBRegressor()

In [14]:
xgbr_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
train_predictions = pd.DataFrame()
train_predictions["True Sales"] = (
    sales_transform
        .inverse_transform(train_df[["Sales"]])
        .squeeze()
)
train_predictions["Predicted Sales"] = (
    sales_transform
        .inverse_transform((xgbr_regressor.predict(X_train))[np.newaxis, :])
        .squeeze()
)

test_predictions = pd.DataFrame()
test_predictions["True Sales"] = (
    sales_transform
        .inverse_transform(test_df[["Sales"]])
        .squeeze()
)
test_predictions["Predicted Sales"] = (
    sales_transform
        .inverse_transform(xgbr_regressor.predict(X_test)[np.newaxis, :])
        .squeeze()
)

display(train_predictions)
display(test_predictions)

,True Sales,Predicted Sales
0,6993.0,6838.243164
1,7014.0,7195.817871
2,18643.0,19076.705078
3,7713.0,7781.482910
4,7869.0,7940.080078
...,...,...
114743,11311.0,11383.973633
114744,8408.0,8315.711914
114745,8649.0,8736.795898
114746,17770.0,17818.517578


,True Sales,Predicted Sales
0,6447.0,6545.398926
1,9586.0,9689.464844
2,11194.0,11102.532227
3,7414.0,7296.965332
4,9293.0,9265.283203
...,...,...
28682,7465.0,7123.370605
28683,12353.0,12644.301758
28684,4707.0,4705.242676
28685,10014.0,10119.153320


In [16]:
train_mae = mean_absolute_error(
    train_predictions["True Sales"],
    train_predictions["Predicted Sales"]
)
train_mse = mean_squared_error(
    train_predictions["True Sales"],
    train_predictions["Predicted Sales"]
)

test_mae = mean_absolute_error(
    test_predictions["True Sales"],
    test_predictions["Predicted Sales"]
)
test_mse = mean_squared_error(
    test_predictions["True Sales"],
    test_predictions["Predicted Sales"]
)

print(f"Forecasting results -- train MAE | MSE: {train_mae:.2f} | {train_mse:.2f} -- test MAE | MSE: {test_mae:.2f} | {test_mse:.2f}")

Forecasting results -- train MAE | MSE: 109.67 | 32390.39 -- test MAE | MSE: 122.78 | 58980.94
